In [ ]:
import statsmodels.stats.api as sms


# Расчитываем необходимый размер выборки для каждой группы
# Базовая конверсия в задании = 9%
# Погрешность 5%
# alpha=5%
baseline_rate = 0.09
minimum_effect = 0.05
alpha = 0.05

def sample_size_calculator(baseline_rate, minimum_effect, power=0.8, alpha=alpha):

    effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + minimum_effect)
    sample_size = sms.NormalIndPower().solve_power(effect_size, power=power, alpha=alpha, ratio=1)
    return sample_size

sample_size = sample_size_calculator(baseline_rate, minimum_effect)
print(f'Необходимый размер выборки для каждой группы: {sample_size:.0f}')

Необходимый размер выборки для каждой группы: 632


In [ ]:
# df.head()

*** ***
 - Первая таблица для обработки лежит здесь

https://drive.google.com/file/d/1oAg3A97c8ASUVizYHvnMFc7T8oWGq1Th/view?usp=sharing

 - Вторая, третья и четвертая таблица для загрузки

https://drive.google.com/file/d/1Sj8CFWxcKFaOivNjXlsJTgeAwyJcW6eP/view?usp=sharing

https://drive.google.com/file/d/1dOD-FaDmQHdC0To7sAahedraO4vQiF9Y/view?usp=sharing

https://drive.google.com/file/d/1D1B19gJcwM_8kKVy_4m_Qu74nL2KspD2/view?usp=sharing

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('ab_activity_3.csv')

# разделяем датасет на контрольную и тестовую группу
test_x = np.array(df[df['fork']=='last-seats']['paid_event'])
control_x = np.array(df[df['fork']=='control']['paid_event'])

# проверяем размер группы
print(f'test group size: {test_x.shape[0]} \ncontrol group size: {control_x.shape[0]} \nrequired group size for each: {sample_size:.0f}')

test group size: 37283 
control group size: 37166 
required group size for each: 632


In [ ]:
import scipy.stats as stats


def c2_test(control_x, test_x):
  control_group = np.array([control_x.sum(), len(control_x)-control_x.sum()])
  test_group = np.array([test_x.sum(), len(test_x)-test_x.sum()])


  # Запускаем тест
  chi2, p, dof, ex = stats.chi2_contingency([control_group, test_group], correction=False)

  p1 = test_group[0] / test_group.sum()
  p2 = control_group[0] / control_group.sum()

  # Расчет отношения конверсии
  conv = p1 / p2

  # Расчет стандартной ошибки для конверсии
  se_p1 = np.sqrt(p1 * (1 - p1) / test_group.sum())
  se_p2 = np.sqrt(p2 * (1 - p2) / control_group.sum())
  std_error = conv * np.sqrt((se_p1 / p1) ** 2 + (se_p2 / p2) ** 2)

  # Доверительный интервал
  ci = stats.norm.interval(0.95, loc=conv, scale=std_error)
  # Выводим результаты
  print("Хи-квадрат p-value: ", p)
  print("Доверительный интервал изменения (conv): ", ci)

  # Проверяем есть ли статистически значимое изменение
  # Т.к. ДИ строился с центром в точке conv(отношение конверсий test/control), значения справа от 1 принадлежат тестовой выборке
  if p < 0.05 and ci[0] > 1:
      print("У тестовой группы конверсия лучше")
  elif p < 0.05 and ci[1] < 1:
      print("У контрольной группы конверсия лучше.")
  else:
      print("Статистически значимой разницы нет.")
  return p1, p2, p, ci

p1, p2, p, ci = c2_test(control_x, test_x)

Хи-квадрат p-value:  0.019966689952271906
Доверительный интервал изменения (conv):  (1.0092405061113718, 1.1354182185637705)
У тестовой группы конверсия лучше


In [ ]:
conv_diff = (p1-p2)*100
print(f'Изменение в конверсии между контрольной и тестовой группой составляют: {conv_diff:.1f}%')

Изменение в конверсии между контрольной и тестовой группой составляют: 0.4%


Согласно проведенному AB тесту, на основе имеющихся данных мы можем отвергнуть нулевю гипотезу, в сторону правосторонней альернативной гипотезы (конверсия тестовой выборки лучше)

Если презентовать материал: мы видим улучшение конверсии на основании имеющихся данных, разница конверсии между тестовой группой и контрольной составила 0.4%, но эти данные не говорят нам о том что это разница конверсий генеральной совокупности, лишь позволяют нам понять что внесенное изменение является статистически значимым и привело к улучшению.

In [ ]:
print('Разница в данных для международных полетов')

df_2 = pd.read_csv('international_flights.csv')
test_y = np.array(df_2[df_2['fork']=='last-seats']['paid_event'])
control_y = np.array(df_2[df_2['fork']=='control']['paid_event'])

p1, p2, p, ci = c2_test(control_y, test_y)
conv_diff = (p1-p2)*100
print(f'Изменение в конверсии между контрольной и тестовой группой составляют: {conv_diff:.1f}%')

Разница в данных для международных полетов
Хи-квадрат p-value:  0.7069216941926919
Доверительный интервал изменения (conv):  (0.828322015422403, 1.1157511019105288)
Статистически значимой разницы нет.
Изменение в конверсии между контрольной и тестовой группой составляют: -0.1%


In [ ]:
print('Разница в данных для внутренних полетов')

df_3 = pd.read_csv('internal_flights.csv')
test_z = np.array(df_3[df_3['fork']=='last-seats']['paid_event'])
control_z = np.array(df_3[df_3['fork']=='control']['paid_event'])

p1, p2, p, ci = c2_test(control_z, test_z)
conv_diff = (p1-p2)*100
print(f'Изменение в конверсии между контрольной и тестовой группой составляют: {conv_diff:.1f}%')

Разница в данных для внутренних полетов
Хи-квадрат p-value:  0.005482682527592331
Доверительный интервал изменения (conv):  (1.0251378871296422, 1.1674825559749569)
У тестовой группы конверсия лучше
Изменение в конверсии между контрольной и тестовой группой составляют: 0.6%


In [ ]:
print('Разница в данных для повторных клиентов')

df_4 = pd.read_csv('travelers.csv')
test_trv = np.array(df_4[(df_4['fork']=='last-seats') & (df_4['had_flights']==1)]['paid_event'])
control_trv = np.array(df_4[(df_4['fork']=='control') & (df_4['had_flights']==1)]['paid_event'])

p1, p2, p, ci = c2_test(test_trv, control_trv)
conv_diff = (p1-p2)*100
print(f'Изменение в конверсии между контрольной и тестовой группой составляют: {conv_diff:.1f}%')

Разница в данных для повторных клиентов
Хи-квадрат p-value:  0.502224887610929
Доверительный интервал изменения (conv):  (0.8717051799561283, 1.062030533141605)
Статистически значимой разницы нет.
Изменение в конверсии между контрольной и тестовой группой составляют: -0.2%


In [ ]:
print('Разница в данных для новых клиентов')

test_Non_trv = np.array(df_4[(df_4['fork']=='last-seats') & (df_4['had_flights']==0)]['paid_event'])
control_Non_trv = np.array(df_4[(df_4['fork']=='control') & (df_4['had_flights']==0)]['paid_event'])

p1, p2, p, ci = c2_test(test_Non_trv, control_Non_trv)
conv_diff = (p1-p2)*100
print(f'Изменение в конверсии между контрольной и тестовой группой составляют: {conv_diff:.1f}%')

Разница в данных для новых клиентов
Хи-квадрат p-value:  0.01450109918571266
Доверительный интервал изменения (conv):  (0.8456277652545003, 0.9795411535195155)
У контрольной группы конверсия лучше.
Изменение в конверсии между контрольной и тестовой группой составляют: -0.5%


Как дополнительные данные мы могли бы использовать выборку с параметром "Кредит одобрен" - для анализа конверсии в покупку.

В идеале на моменте сбора информации вытащить из логов данные "Осталось билетов", для анализа информации о том, насколько вырос спрос на кредитную услугу для срочной покупки авиабилета, например при параметре "Осталось авиабилетов<=4"